In [3]:
# !pip install pandas
# !pip install transformers
# !pip install scikit-learn
import sys
sys.path.append("/home/pj/.local/lib/python3.10/site-packages")
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer

import torch

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification



ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [2]:
data = pd.read_csv("C:\\Users\\ASUS\\Desktop\\EmotionDetection.csv")

In [3]:

data = data.drop_duplicates()

data['text'] = data['text'].str.lower().str.strip()

label_map = {label:idx for idx, label in enumerate(data['Emotion'].unique())}

data['label'] = data['Emotion'].map(label_map)

In [4]:
train, temp = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

val, test = train_test_split(temp, test_size=0.5, stratify=temp['label'], random_state=42)

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Number of classes, e.g., emotions
num_labels = len(set(train['label']))

# Load DistilBERT model with classification head
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model check

In [6]:
def tokenize(texts):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=128, return_tensors='pt')

train_encodings = tokenize(train['text'])
val_encodings = tokenize(val['text'])
test_encodings = tokenize(test['text'])

In [7]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, train['label'].tolist())
val_dataset = EmotionDataset(val_encodings, val['label'].tolist())
test_dataset = EmotionDataset(test_encodings, test['label'].tolist())

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="C:\\Users\\ASUS\\Desktop\\DataFloat\\Model",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    fp16=True,  # Enable mixed precision (faster training if GPU supports it)
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
